In [2]:
# Check if GPU active
!nvidia-smi

Wed Mar  4 01:36:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import pandas as pd
import numpy as np

In [3]:

data = "https://raw.githubusercontent.com/spotify-song-suggester-6/data-science/develop/rawData/SpotifyAudioFeaturesApril2019.csv"

df = pd.read_csv( data)

df.head()

,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.005820,0.743,238373,0.339,0.000,1,0.0812,-7.678,1,0.4090,203.927,4,0.118,15
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.024400,0.846,214800,0.557,0.000,8,0.2860,-7.259,1,0.4570,159.009,4,0.371,0
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,0.025000,0.603,138913,0.723,0.000,9,0.0824,-5.890,0,0.0454,114.966,4,0.382,56
3,Chris Cooq,3J2Jpw61sO7l6Hc7qdYV91,Lactose,0.029400,0.800,125381,0.579,0.912,5,0.0994,-12.118,0,0.0701,123.003,4,0.641,0
4,Chris Cooq,2jbYvQCyPgX3CdmAzeVeuS,Same - Original mix,0.000035,0.783,124016,0.792,0.878,7,0.0332,-10.277,1,0.0661,120.047,4,0.928,0


In [3]:
df.shape

(130663, 17)

In [0]:

x = df.drop( columns= {'track_id', 'artist_name', 'track_name', 'duration_ms'})
y = df[ 'track_id']

In [4]:

df.iloc[2526]

artist_name                                           Antonio Vivaldi
track_id                                       2u2L0LxJ2aEOEjYDJ90Tpo
track_name          The Four Seasons, Op. 8: Violin Concerto in F ...
acousticness                                                    0.756
danceability                                                    0.177
duration_ms                                                    121707
energy                                                         0.0447
instrumentalness                                               0.0882
key                                                                 5
liveness                                                       0.0858
loudness                                                      -27.245
mode                                                                1
speechiness                                                    0.0402
tempo                                                          70.013
time_signature      

In [5]:

sample = df.drop( columns= {'track_id', 'artist_name', 'track_name', 'duration_ms'}).iloc[2526]

sample

acousticness         0.7560
danceability         0.1770
energy               0.0447
instrumentalness     0.0882
key                  5.0000
liveness             0.0858
loudness           -27.2450
mode                 1.0000
speechiness          0.0402
tempo               70.0130
time_signature       3.0000
valence              0.0361
popularity          14.0000
Name: 2526, dtype: float64

In [0]:
## STRAIGHT KNN ATTEMPT:

In [0]:
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [7]:

xTrain, xTest, yTrain, yTest = train_test_split( x, y, test_size= .2)

xTrain.shape, xTest.shape, yTrain.shape, yTest.shape

((104530, 13), (26133, 13), (104530,), (26133,))

In [9]:

knn = KNeighborsClassifier( n_neighbors= 8,   
                            weights= 'uniform',  
                            algorithm= 'auto', 
                            leaf_size= 32,
                            p= 2, 
                            n_jobs= -1
)
knn.fit( xTrain, yTrain)

KNeighborsClassifier(algorithm='auto', leaf_size=32, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=8, p=2,
                     weights='uniform')

In [10]:

# After fitting, run prediction on xTest subset, and return top 10?
results = knn.predict( xTest)[0:10]

print( results)

['0NLppQWVulM7aVODrrimrW' '3HytV6KvmMlb7m9h5Ku492'
 '13uBL8TBoO7ThME4EB4sMT' '0afqYxxJH8D0ZYgMEyD2xb'
 '1ZLY3ZTRwW7vNjcm50wvkT' '07HU9ata6MbzBFMOy43xAJ'
 '013TFRzJWkT41RRSgSenfI' '0xg7HkBSt47X7Fj8bztbqo'
 '0VqRfTR1NUPAFbikGN08N8' '1BCkIYJWvhekqxF8NCGfgp']


In [11]:
knn.score( xTest, yTest)

0.0007270500899246164

In [13]:
cvTest = cross_val_score( knn, x, y, cv= 3)
print( cvTest)
print( 'mean: {:.5f}%'.format( np.mean( cvTest) *100))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


[0.00066582 0.00066584 0.00064288]
mean: 0.06582%


In [14]:

knn = KNeighborsClassifier( n_neighbors= 16,   
                            weights= 'distance',  
                            algorithm= 'kd_tree',  
                            leaf_size= 256,
                            p= 2, 
                            n_jobs= -1
)
knn.fit( xTrain, yTrain)

KNeighborsClassifier(algorithm='kd_tree', leaf_size=256, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=16, p=2,
                     weights='distance')

In [16]:

# After fitting, run prediction on xTest subset, and return top 10?
results = knn.predict( xTest)[0:10]

print( results)

['3yrjmCDexSiDWCmvozHuHC' '6t2QY6vgSrXpJRBY4EQkrL'
 '3RK60inxAxrJVdZtdqRjNt' '4UeM1bbFkRfJZ5BnwQ2OvC'
 '6uzv7ZcwreLiA7DSS3Ju38' '7Ft1qhUD42q8uRpkxpOClo'
 '013TFRzJWkT41RRSgSenfI' '2a9w6WnGmcFify7nCR5j2B'
 '6iGAk5TizZBLkduf0KQDPD' '5N52lSaVZ5aG4YuXu7pTzJ']


In [17]:
knn.score( xTest, yTest)

0.0042857689511345805

In [18]:
cvTest = cross_val_score( knn, x, y, cv= 3)
print( cvTest)
print( 'mean: {:.5f}%'.format( np.mean( cvTest) *100))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


[0.00507404 0.0049364  0.00518896]
mean: 0.50665%


In [18]:
results = knn.predict( sample)[0:10]

print( results)

['30onrKEgOU8OGXbyF7zLyI']


In [0]:

################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################

In [0]:
## Unknown to me at the moment, how to get the data I need from standard KNN
## --> only able to get 10 songs out when > 10 are given as input??
## Switching to discovered alternative, KDTree

In [0]:
from sklearn.neighbors import KDTree

In [53]:
type( sample), type( x.sample(1))

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

In [62]:
kdt = KDTree( x, leaf_size= 2048)
dist, ind = kdt.query( sample, k= 11)

print( ind)
print( dist)

[[  2526 110620 114268  85896 114857  66115  17303  53329  92928 110223
  117042]]
[[0.         2.33493907 2.61164687 2.63379147 2.75688268 2.8232957
  2.82474875 2.8671262  2.91112742 3.06422877 3.1866923 ]]


In [66]:
df.iloc[ind[0][1:11]]

,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
110620,Wolfgang Amadeus Mozart,2BK7DJt8evygm5Qmvm9Pxv,"Symphony No. 19 in E-Flat Major, K. 132: II. A...",0.987,0.1400,227472,0.02200,0.428,6,0.1260,-26.045,1,0.0499,68.336,3,0.2010,14
114268,Claude Debussy,4v0U45p6YpkfBD6rWcsmhQ,"Images oubliées, L. 87: 1. Lent (mélancolique ...",0.994,0.4200,210027,0.00783,0.890,4,0.0621,-28.823,1,0.0493,69.258,4,0.0393,13
85896,Hedley Patel,0dAwTbkidofxjmFC9XerzL,Red,0.993,0.3570,119456,0.07410,0.904,7,0.1170,-26.893,0,0.0567,69.885,4,0.2370,14
114857,Wolfgang Amadeus Mozart,7LGh7X1IqvnOMIEblrSlV0,"Piano Concerto No. 16 in D Major, K. 451: II. ...",0.990,0.3580,351000,0.02280,0.891,7,0.1000,-27.990,1,0.0371,70.566,4,0.1190,13
66115,Binaural Beats Sleep,1EzkyMQ6ubKUatNxX0FSPB,Brainwave Entrainment for Deep Sleep,0.984,0.1360,147600,0.04300,0.920,4,0.0985,-27.180,0,0.0455,71.118,3,0.0358,16
17303,Warmth,21l7beFvSCpuPsqcM5bY3q,Parallel,0.931,0.0633,452813,0.18300,0.976,6,0.0971,-25.101,1,0.0456,69.284,3,0.0379,13
53329,Study Music & Sounds,7H6ziUVrb4Yt4k4nK9LbDh,Study Alpha Waves (Thunderstorm),0.975,0.1760,121190,0.21600,0.967,6,0.1090,-26.075,1,0.0410,70.050,4,0.0392,16
92928,Johann Sebastian Bach,3KcIGNhLYEPidm6gZn9Uhw,"Christ, der du bist der helle Tag, BWV 766: I....",0.954,0.2350,49827,0.00652,0.988,5,0.1120,-28.333,0,0.0364,69.364,4,0.1600,12
110223,Claude Debussy,0X6Njzx9qBmjrmCAnPepOC,"Sonata for Flute, Viola and Harp in F Major, L...",0.989,0.2780,415653,0.02530,0.447,5,0.0831,-26.621,1,0.0450,67.219,4,0.0433,14
117042,Claude Debussy,1FI5X8zFVFHuNPRUDdtCZd,"Préludes, Book 1, L. 117: No. 1, Danseuses de ...",0.996,0.5080,142173,0.00751,0.923,5,0.0911,-28.384,1,0.1020,72.460,4,0.0605,13


In [63]:
df.iloc[ind[0][1:10]]

,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
110620,Wolfgang Amadeus Mozart,2BK7DJt8evygm5Qmvm9Pxv,"Symphony No. 19 in E-Flat Major, K. 132: II. A...",0.987,0.1400,227472,0.02200,0.428,6,0.1260,-26.045,1,0.0499,68.336,3,0.2010,14
114268,Claude Debussy,4v0U45p6YpkfBD6rWcsmhQ,"Images oubliées, L. 87: 1. Lent (mélancolique ...",0.994,0.4200,210027,0.00783,0.890,4,0.0621,-28.823,1,0.0493,69.258,4,0.0393,13
85896,Hedley Patel,0dAwTbkidofxjmFC9XerzL,Red,0.993,0.3570,119456,0.07410,0.904,7,0.1170,-26.893,0,0.0567,69.885,4,0.2370,14
114857,Wolfgang Amadeus Mozart,7LGh7X1IqvnOMIEblrSlV0,"Piano Concerto No. 16 in D Major, K. 451: II. ...",0.990,0.3580,351000,0.02280,0.891,7,0.1000,-27.990,1,0.0371,70.566,4,0.1190,13
66115,Binaural Beats Sleep,1EzkyMQ6ubKUatNxX0FSPB,Brainwave Entrainment for Deep Sleep,0.984,0.1360,147600,0.04300,0.920,4,0.0985,-27.180,0,0.0455,71.118,3,0.0358,16
17303,Warmth,21l7beFvSCpuPsqcM5bY3q,Parallel,0.931,0.0633,452813,0.18300,0.976,6,0.0971,-25.101,1,0.0456,69.284,3,0.0379,13
53329,Study Music & Sounds,7H6ziUVrb4Yt4k4nK9LbDh,Study Alpha Waves (Thunderstorm),0.975,0.1760,121190,0.21600,0.967,6,0.1090,-26.075,1,0.0410,70.050,4,0.0392,16
92928,Johann Sebastian Bach,3KcIGNhLYEPidm6gZn9Uhw,"Christ, der du bist der helle Tag, BWV 766: I....",0.954,0.2350,49827,0.00652,0.988,5,0.1120,-28.333,0,0.0364,69.364,4,0.1600,12
110223,Claude Debussy,0X6Njzx9qBmjrmCAnPepOC,"Sonata for Flute, Viola and Harp in F Major, L...",0.989,0.2780,415653,0.02530,0.447,5,0.0831,-26.621,1,0.0450,67.219,4,0.0433,14


In [65]:
import joblib

filename = 'model-v1.sav'

joblib.dump( kdt, filename)



['model-v1.sav']

In [0]:
##############################################################################################
##############################################################################################
#                                                                                            #
#      ###   ##      ###   ##        ########      ########      ###   ##      ########      #
#      ####  ##      ####  ##             ###      ##    ##      ####  ##      ##            #
#      ## ## ##      ## ## ##          ####        ##    ##      ## ## ##      #####         #
#      ##  ####      ##  ####        ###           ##    ##      ##  ####      ##            #
#      ##   ###      ##   ###        ########      ########      ##   ###      ########      #
#                                                                                            #
##############################################################################################
##############################################################################################

In [0]:
!pip install tensorflow --upgrade

from tensorflow import convert_to_tensor
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Embedding, Dropout, Activation
from tensorflow.keras.layers import Input, LSTM
from tensorflow.keras.optimizers import SGD, Adam
from keras.preprocessing import sequence

from sklearn.preprocessing import StandardScaler

In [0]:
epochs = 2
batches = 2
steps = 50000
encoding_dim = 10

In [0]:
# LSTM (NYI)
def buildModel1():
    return Sequential(
            [Embedding( 14, 512),
             LSTM( 256, dropout= 0.3, recurrent_dropout= 0.3),
             LSTM( 64, dropout= 0.3, recurrent_dropout= 0.3),
             Dense( 10, activation= 'relu')]
    )

In [0]:
# Sequential 
def buildModel2():
    return Sequential(
            [#Dense( 64, input_shape= (14,)),
#             Activation( "relu"),
#             Dropout( .25),
#             Dense( 128),
#             Activation( "relu"),
#             Dropout( .25),
             Dense( 10, input_shape= (14,)),
             Activation( "sigmoid")]
    )

In [24]:
print('First, we Build the model...\n')
spootify = buildModel2()
print( spootify.summary(), '\n')
spootify.compile( loss= 'sparse_categorical_crossentropy',
                  optimizer= 'adam',
                  metrics= ['accuracy']
)

First, we Build the model...

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                960       
_________________________________________________________________
activation_4 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                650       
_________________________________________________________________
activation_5 (Activation)    (None, 10)                0         
Total params: 1,610
Trainable params: 1,610
Non-trainable params: 0
_________________________________________________________________
None 



In [0]:
print('Then, we Train the model...\n')
spootify.fit( xTrain, yTrain,
              batch_size= batches,
              epochs= epochs,
              verbose= 2,
              validation_data= (xTest, yTest)
)

In [0]:
whatScorePrecious, whatAccuracy = spootify.evaluate( xTest, yTest, 
                                                     batch_size= 64
)
print('Test score:', whatScorePrecious)
print('Test accuracy:', whatAccuracy)

In [0]:
moodel = Sequential()
moodel.add( Dense( 10, input_shape= (14,)))
moodel.add( Activation( "sigmoid"))
#moodel.add( Dropout( .25))
#moodel.add( Dense( 2))
#moodel.add( Activation( "softmax"))

spudify.compile( loss= "sparse_categorical_crossentropy",
                optimizer= "sgd",
                metrics= ["accuracy"]
)

fitBit = spudify.fit( xTrain, yTrain,
                     batch_size= batches,
                     epochs= epochs,
                     verbose= 10,
                     validation_data= ( xTest, yTest)
)

In [9]:
inputFrame = Input( shape= (14,))

encoded = Dense( 256, activation= 'relu')(inputFrame)
encoded = Dense( 64, activation= 'relu')(encoded)
encoded = Dense( 10, activation= 'relu')(encoded)

decoded = Dense( 64, activation= 'relu')(encoded)
decoded = Dense( 256, activation= 'relu')(decoded)
decoded = Dense( 14, activation= 'sigmoid')(decoded)

spautofy = Model(inputFrame, decoded)

spautofy.compile( optimizer= 'adam',
                  loss= 'kld',
                  metrics= ['accuracy']
)



spautofy.fit( xTrain, xTrain,
              epochs= 32,
              batch_size= 256,
              shuffle= True,
              validation_data= (xTest, xTest),
              verbose= True
)

Train on 104530 samples, validate on 26133 samples
Epoch 1/32
104530/104530 [==============================] - 2s 21us/sample - loss: 12.1596 - accuracy: 0.0000e+00 - val_loss: 10.0466 - val_accuracy: 0.0000e+00
Epoch 2/32
104530/104530 [==============================] - 2s 16us/sample - loss: 10.1037 - accuracy: 0.0000e+00 - val_loss: 10.0466 - val_accuracy: 0.0000e+00
Epoch 3/32
104530/104530 [==============================] - 2s 16us/sample - loss: 10.1037 - accuracy: 0.0000e+00 - val_loss: 10.0466 - val_accuracy: 0.0000e+00
Epoch 4/32
104530/104530 [==============================] - 2s 16us/sample - loss: 10.1037 - accuracy: 0.0000e+00 - val_loss: 10.0466 - val_accuracy: 0.0000e+00
Epoch 5/32
104530/104530 [==============================] - 2s 16us/sample - loss: 10.1037 - accuracy: 0.0000e+00 - val_loss: 10.0466 - val_accuracy: 0.0000e+00
Epoch 6/32
104530/104530 [==============================] - 2s 16us/sample - loss: 10.1037 - accuracy: 0.0000e+00 - val_loss: 10.0466 - val_accu

In [0]:
spautofy.reset_metrics()

decodedFrame = spautofy.predict( xTest)

In [22]:
import joblib

filename = 'model(v0).sav'

joblib.dump( decodedFrame, filename)

['model(v0).sav']

In [0]:
loadedModel = joblib.load( filename)

## TODO: use this to output 10 results (is that even possible??)

In [0]:
## STRETCH?

In [0]:
from time import time
import keras.backend as K
from keras.engine.topology import Layer, InputSpec
from keras import callbacks
from keras.initializers import VarianceScaling
from sklearn.cluster import KMeans
from sklearn import metrics
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import SGD

In [0]:
# HYPERPARAMS

DIMS = [x.shape[-1], 500, 500, 2000, 10]
INIT = VarianceScaling( scale= 1. / 3., mode= 'fan_in', 
                        distribution= 'uniform')
PRETRAIN_OPT = SGD( lr= 1, momentum= 0.9)
PRETRAIN_EPOCHS = 300
BATCH = 256

In [0]:

data = "https://raw.githubusercontent.com/spotify-song-suggester-6/data-science/develop/rawData/SpotifyAudioFeaturesApril2019.csv"

df = pd.read_csv( data)

x = df.drop( columns= {'track_id', 'artist_name', 'track_name', 'duration_ms'})
y = df[ 'track_id']

nClusters = len( np.unique( y))
x.shape

In [0]:

def autoEnfolder( dims, act= 'relu', init= 'glorot_uniform'):
    """
    Fully connected, symmetric auto-encoder model
    Arguments:
        dims: list of number of units in each layer of encoder
        dims[0] is input dim 
        dims[-1] is units in hidden layer
            Decoder is symmetric with encoder, so number of layers of autoencoder is 2*len(dims)-1
        act: activation, not applied to Input, Hidden, or Output layers
    return:
        (ae_model, encoder_model), Model of autoencoder and model of encoder

    """


    nStacks = len( dims) - 1
    inputImg = Input( shape=( dims[0],), name= 'input')
    x = inputImg
    
    # internal encoder layers
    for i in range( nStacks-1):
        x = Dense( dims[ i + 1], activation= act, 
                   kernel_initializer= init, name= 'encoder_%d' % i)(x)

    # hidden layer
    encoded = Dense( dims[-1], kernel_initializer= init, 
                     name= 'encoder_%d' % (n_stacks - 1))(x)  # hidden layer, features are extracted from here
    x = encoded

    # internal layers in decoder
    for i in range( nStacks-1, 0, -1):
        x = Dense( dims[i], activation= act, kernel_initializer= init, 
                   name= 'decoder_%d' % i)(x)

    # output
    x = Dense( dims[0], kernel_initializer= init, name= 'decoder_0')(x)
    dexploded = x
    return Model( inputs= inputImg, 
                  outputs= dexploded, name= 'AE'), 
           Model( inputs= inputImg, 
                  outputs= encoded, name= 'encoder')

In [0]:

class ClusteringLayer( Layer):
    """
    Clustering layer converts input sample (feature) to soft label

    # Example
    ```
        model.add( ClusteringLayer( n_clusters= 10))

    ```

    # Arguments
        n_clusters: number of clusters
        weights: list of numpy array with shape (n_clusters, n_features), representing initial cluster centers
        alpha: degrees of freedom parameter in student t-distribution. Default  1
    # Input shape
        2D tensor with shape (n_samples, n_features)
    # Output shape
        2D tensor with shape (n_samples, n_clusters)

    """

    def __init__( self, nClusters, weights= None, alpha= 1.0, **kwargs):
        if 'input_shape' not in kwargs and 'inputDim' in kwargs:
            kwargs[ 'input_shape'] = (kwargs.pop( 'inputDim'),)
        super( ClusteringLayer, self).__init__( **kwargs)
        self.nClusters = nClusters
        self.alpha = alpha
        self.initialWeights = weights
        self.inputSpec = InputSpec( ndim= 2)

    def build( self, inputShape):
        assert len( inputShape) == 2
        inputDim = inputShape[1]
        self.inputSpec = InputSpec( dtype= K.floatx(), shape= (None, inputDim))
        self.clusters = self.add_weight( (self.nClusters, inputDim), initializer= 'glorot_uniform', name= 'clusters')
        if self.initialWeights is not None:
            self.setWeights( self.initialWeights)
            del self.initialWeights
        self.built = True

    def call( self, inputs, **kwargs):
        """ 
        student t-distribution, same as used in t-SNE algorithm        
                 q_ij = 1/(1+dist( x_i, µ_j) ^2), then normalize
                 q_ij can be interpreted as probability of assigning sample i to cluster j
                 (e.g., a soft assignment)
        Arguments:
            inputs: variable containing data, shape= (n_samples, n_features)
        Return:
            q: student t-distribution, or soft labels for each sample. shape= (n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum( K.square( K.expand_dims( inputs, axis= 1) - self.clusters), axis= 2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose( K.transpose( q) / K.sum( q, axis= 1)) # Make sure each sample's 10 values add up to 1
        return q

    def computeOutputShape( self, inputShape):
        assert inputShape and len( inputShape) == 2
        return inputShape[0], self.nClusters

    def getConfig( self):
        config = {'nClusters': self.nClusters}
        baseConfig = super( ClusteringLayer, self).getConfig()
        return dict( list( baseConfig.items()) + list( config.items()))

In [0]:

autoExploder, encoder = autoEnfolder( DIMS, init= INIT)

autoExploder.compile( optimizer= PRETRAIN_OPT, loss= 'mse')
autoExploder.fit( x, x, batch_size= BATCH, epochs= PRETRAIN_EPOCHS)

In [0]:
# 
# kayMe = KMeans( n_clusters= nClusters, n_init= 20, n_jobs= -1)
# yPredK = kayMe.fit_predict( x)
# 
# metrics.acc( y, yPredK)

In [0]:
clusteringLayr = ClusteringLayer( nClusters, name= 'clustering')(encoder.output)
model = Model( inputs= encoder.input, outputs= clusteringLayr)

In [0]:

# Initialize cluster centers using k-means
kayMe = KMeans( n_clusters= nClusters, n_init= 20, n_jobs= -1)
yPred = kayMe.fit_predict( encoder.predict( x))

yPredLast = np.copy( yPred)

model.get_layer( name= 'clustering').set_weights( [kayMe.cluster_centers_])


def target_distribution( q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

In [0]:
model.compile( optimizer= SGD( 0.01, 0.9), loss= 'kld')

loss, index = 0, 0
indArray = np.arange( x.shape[0])
maxIter = 8000
updateInterval = 140
tolerance = .001        # tolerance threshold to stop training

for items in range( int( maxIter)):
    if items % updateInterval == 0:
        q = model.predict( x, verbose= 0)
        p = target_distribution( q)  # update aux target distribution p
        
        # eval clustering performance
        yPred = q.argmax( 1)
        if y is not None:
            acc = np.round( metrics.acc( y, yPred), 5)

    idx = indArray[ index * BATCH: min( (index+1) * BATCH, x.shape[0])]
    loss = model.train_on_batch( x= x[ idx], y= p[ idx])
    index = index + 1 if (index + 1) * BATCH <= x.shape[0] else 0

In [0]:
# POSSIBLE TO SWAP KMEAN FOR KDTREE IF DOESN'T WORK??
#
# kdt = KDTree( x, leaf_size= 2048)
# dist, ind = kdt.query( sample, k= 11)
# 
# print( ind)
# print( dist)

In [0]:
import joblib

filename = 'stretchModel.sav'

joblib.dump( kdt, filename)

In [0]:
import seaborn as sns
import sklearn.metrics
import matplotlib.pyplot as plt
sns.set( font_scale= 3)
confucius = sklearn.metrics.confusion_matrix( y, yPred)

plt.figure( figsize= (16, 14))
sns.heatmap( confucius, annot= True, fmt= "d", annot_kws= {"size": 20});
plt.title( "Confusion matrix", fontsize= 30)
plt.ylabel( 'True label', fontsize= 25)
plt.xlabel( 'Clustering label', fontsize= 25)
plt.show()